In [2]:
from sources.normalizer.news_normalizer import NewsNormalizer
from sources.saver.file_saver import FileSaver
from sources.annotator.xml_annotator import XMLAnnotator
from sources.source.suspilne_source import SuspilneSource
from sources.parser.suspilne_parser import SuspilneParser

corpus_path = "corpus"

for record in SuspilneSource.records():
    title = record.metadata.title
    
    print(f"Processing: {title}")
    content = record.content()

    annotator = XMLAnnotator()
    document = SuspilneParser.parse(content, metadata=record.metadata,annotator=annotator)
    base_text = annotator.get_string(document)
    
    saver = FileSaver(path=f"{corpus_path}/base")
    saver.save(base_text, name=f"{title}.txt")
    
    warnings = []
    errors = []
    
    def normalize(text: str) -> str:
        new_text, new_warnings, new_errors = NewsNormalizer.normalize(text)

        warnings.extend(new_warnings)
        errors.extend(new_errors)
        
        return new_text
    
    document = annotator.update_elements(document, normalize)
    
    annotator.add_warnings(document, warnings)
    annotator.add_errors(document, errors)
    normalized_text = annotator.get_string(document)

    saver = FileSaver(path=f"{corpus_path}/normalized")
    saver.save(normalized_text, name=f"{title}.txt")
    
    
    print(f"Finished: {title}")





Processing: Як росіяни збирають інформацію про військовослужбовців з Полтавщини
Finished: Як росіяни збирають інформацію про військовослужбовців з Полтавщини
Processing: Найбільші країни ЄС не підтримують пропозицію виділити 20 млрд пакет допомоги Україні — топдипломат ЄС
Finished: Найбільші країни ЄС не підтримують пропозицію виділити 20 млрд пакет допомоги Україні — топдипломат ЄС
Processing: Весняний ярмарок, день відкритих дверей і виготовлення окопних свічок: куди піти в Миколаєві вихідними
Finished: Весняний ярмарок, день відкритих дверей і виготовлення окопних свічок: куди піти в Миколаєві вихідними
